# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 5

### **Vectores Embebidos Pre-entrenados: Fasttext**

#### **Nombres y matrículas de los integrantes del equipo:**



*   Elemento de lista
*   Elemento de lista



In [2]:
# Aquí deberás incluir todas las librerías que requieras durante esta actividad:
import pandas as pd  
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer

import re
import string


##**Pregunta - 1:**



Descarga los 3 archivos de Canvas y genera un nuevo DataFrame de Pandas con ellos.

**Llama simplemente "df" a dicho DataFrame.**

Los archivos los encuentras en Canvas: amazon5.txt, imdb5.txt, yelp5.txt.



In [3]:

# ******* Se revisan los 3 dataframes para verificar el numero de registros : ***********


dfa = pd.read_csv('amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('imdb5.txt', delimiter='\t', names=['review','label'], header=None, encoding='utf-8')
dfy = pd.read_csv('yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')


print('Total de registros de Amazon:',dfa.shape)
print('Total de registros de IMBD:',dfi.shape)
print('Total de registros de Yelp:',dfy.shape)

# *********** Aquí termina la sección de agregar código *************


Total de registros de Amazon: (1000, 2)
Total de registros de IMBD: (1000, 2)
Total de registros de Yelp: (1000, 2)


In [4]:
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [5]:
dfi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  1000 non-null   object 
 1   label   0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 15.8+ KB


Se observa que el dfi no realizo bien la separacion entre caracteres y valores. Se tiene que realizar un ajuste.

In [6]:
with open("imdb5.txt","r") as file:
    temp_reviews_imbd=file.read()

reviews_imbd= temp_reviews_imbd.split("\n")

In [7]:
print(reviews_imbd[1:3])

['Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.    0', 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.    0']


In [8]:
print(reviews_imbd[17:19])

["I'll put this gem up against any movie in terms of screenplay, cinematography, acting, post-production, editing, directing, or any other aspect of film-making.    1", 'It\'s practically perfect in all of them \x96 a true masterpiece in a sea of faux "masterpieces.    1']


In [9]:
lista_opiones=[]
lista_valores=[]
for i in reviews_imbd:
    if i.strip():
        i_with_tabs = re.sub(r' {3,}', '\t', i)
        separador=i_with_tabs.split("\t")
        opinion=separador[0]
        valor=separador[1]

        lista_opiones.append(opinion)
        lista_valores.append(float(valor))

dfi = pd.DataFrame({'review': lista_opiones, 'label': lista_valores})

In [10]:
dfi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  1000 non-null   object 
 1   label   1000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 15.8+ KB


In [11]:
dfi.head()

,review,label
0,"A very, very, very slow-moving, aimless movie ...",0.0
1,Not sure who was more lost - the flat characte...,0.0
2,Attempting artiness with black & white and cle...,0.0
3,Very little music or anything to speak of.,0.0
4,The best scene in the movie was when Gerardo i...,1.0


In [12]:
dfy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [13]:
#Se juntan los 3 dataframes en uno solo 
df= pd.concat([dfa, dfi, dfy])
print("Total de registros es " + str(len(df)))

Total de registros es 3000


In [14]:
df.head(2)

,review,label
0,So there is no way for me to plug it in here i...,0.0
1,"Good case, Excellent value.",1.0


In [15]:
# Verifiquemos la información del DataFrame:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  3000 non-null   object 
 1   label   3000 non-null   float64
dtypes: float64(1), object(1)
memory usage: 70.3+ KB


In [16]:
# Y veamos sus primeros registros:

df.head()

,review,label
0,So there is no way for me to plug it in here i...,0.0
1,"Good case, Excellent value.",1.0
2,Great for the jawbone.,1.0
3,Tied to charger for conversations lasting more...,0.0
4,The mic is great.,1.0


##**Pregunta - 2:**

Realiza el proceso de limpieza.

Aplica el preprocesamiento que consideres adecuado, sin embargo, deberás aplicar necesariamente alguna de las técnicas de lematización.











GENERAMOS LA TOKENIZACION DE LOS DATOS

In [17]:
X = df.review     # Serie de strings
Y = df.label      # Serie de enteros 0s y 1s

In [18]:
nltk.download('stopwords')  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/josedejesuspena/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
mis_stopwords=[]
print(len(stopwords.words('english')))
print(stopwords.words('english'))
mis_stopwords=stopwords.words('english')

179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than

In [20]:
negwords = [ 'no', 'nor', 'not', 'ain', 'aren', "aren't", 'don', "don't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [21]:
mystopwords = [i for i in mis_stopwords if i not in negwords]

In [22]:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


def clean_tok(doc):
  

  # Reemplazo de caracteres no alfabéticos con espacios en blanco
    doc = [re.sub(r'[^a-zA-Z\s]', ' ', review) for review in doc]
    
    # Conversión a minúsculas
    doc = [review.lower() for review in doc]
    
    # Eliminación de stopwords
    doc = [[word for word in review.split() if word not in mystopwords] for review in doc]

    #Delimitar la longitud de tokens validos
    long_token = 1
    tokens = [[word for word in sublist if len(word) > long_token] for sublist in doc]


    return tokens


# *********** Aquí termina la sección de agregar código *************

In [23]:
Xcleantok=clean_tok(X)

In [24]:
for x in Xcleantok[0:5]:
  print(x)

['no', 'way', 'plug', 'us', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tied', 'charger', 'conversations', 'lasting', 'minutes', 'major', 'problems']
['mic', 'great']


In [25]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/josedejesuspena/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
from nltk.stem import WordNetLemmatizer
def clean_doc(doc):
    
    
    # Inicializar el lematizador
    lemmatizer = WordNetLemmatizer() 
    
    # Aplicar stemming a cada palabra en cada sublista de doc y mantener las sublistas
    lemmatizer_doc = [[lemmatizer.lemmatize(word) for word in sublist] for sublist in doc]

    #Eliminar las palabras repitas, si existen 
    lemmatizer_sin_repeticiones=[]

    for i in range(len(lemmatizer_doc)):
      sublista_aux=[]
      sublista_temp=lemmatizer_doc[i]
      for sub_i in range(len(sublista_temp)):
        string_temp=sublista_temp[sub_i]
        if string_temp not in sublista_aux:
            sublista_aux.append(string_temp)
      lemmatizer_sin_repeticiones.append(sublista_aux)
    

    return lemmatizer_sin_repeticiones

In [27]:
Xclean = clean_doc(Xcleantok)

In [28]:
len(Xclean)

3000

In [29]:
# Despleguemos los primeros comentarios después de tu proceso de limpieza:

for x in Xclean[0:5]:
  print(x)


['no', 'way', 'plug', 'u', 'unless', 'go', 'converter']
['good', 'case', 'excellent', 'value']
['great', 'jawbone']
['tied', 'charger', 'conversation', 'lasting', 'minute', 'major', 'problem']
['mic', 'great']


#**Pregunta - 3:**



Realicemos una partición aleatoria con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba, respectivamente.

In [30]:
from sklearn.model_selection import train_test_split
# ************* Inicia la sección de agregar código:*****************************

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=1) 
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=17)


# *********** Termina la sección de agregar código *************


# verificemos las dimensiones obtenidas:
print('X,y Train:', len(x_train), len(y_train))
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


#**Pregunta - 4:**



Construye tu vocabulario a continuación


In [31]:
from collections import Counter

In [32]:
# a.	Usa el conjunto de entrenamiento para generar tu vocabulario
#     con un tamaño que consideres adecuado:


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


midiccionario = Counter()    

for k in range(len(x_train)):
  midiccionario.update(x_train[k])

# *********** Aquí termina la sección de agregar código *************

In [33]:
# b.	Indica el tamaño del vocabulario generado.

print('Longitud del vocabulario:', len(midiccionario))  
print('\n(word,frequency):') 
print(midiccionario.most_common(10)) 


# ******* Inicia la sección de agregar código: ***********


None


# *********** Aquí termina la sección de agregar código *************

Longitud del vocabulario: 3684

(word,frequency):
[('not', 205), ('good', 157), ('great', 132), ('movie', 130), ('phone', 123), ('film', 121), ('time', 99), ('one', 93), ('service', 85), ('work', 84)]


In [34]:
#Reducimos el vocuabulario a un mayor frecuencia de aparacion por token.
#Usaremos solo el set de x_train para reducir el vocabulario
#Para que el token aparezca en el vobulario tiene que aparecer al menos 5 veces.


#Se guardaran en una lista todos los token que veamos en xtrain
lista_x_tokens=[]
#mi_dict guarda el numero de indicidencias por token acutal
mi_dict={}
#mi_vocabulario guardara el nuevo vocabulario si el token aparece al menos 5 veces
mi_vocabulario={}
#Frecuencia maxima de aparicion 5 veces
freq_min=5

#Se crea una lista con todos los token del set Xtrain
for i in x_train:
    for token in i:
        lista_x_tokens.append(token)

#Se crea un diccionario apartir del numero de incidencias por token
for i in lista_x_tokens:
    if i not in mi_dict:
        mi_dict[i]=1
    else:
        mi_dict[i]+=1

#Se crea un segundo diccionario (vocabulario) que solo guardara aquellos token que aparecen mayor a 5
for token,freq_actual in mi_dict.items():
    if freq_actual>freq_min:
        mi_vocabulario[token]=freq_actual

print("Para que el token sea valido debe aparecer 5 veces.") 
print("La longitud del nuevo vocabulario es de " + str (len(mi_vocabulario)))

Para que el token sea valido debe aparecer 5 veces.
La longitud del nuevo vocabulario es de 460


c.	¿Por qué debe usarse solamente el conjunto de entrenamiento para generar el vocabulario?


### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


In [35]:
# d.	Con el vocabulario generado, filtra los conjuntos de entrenamiento,
#     validación y prueba para que todos los comentarios usen solamente las
#     palabras de este vocabulario.

#     Llamar train_x, val_x y test_x a estos tres conjuntos.


# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

#Lista que guardara los tokens del vocabulario
tokens_mi_voc=[]


#Guardamos en la lista "tokens_mi_voc" los tokens de mi vocabulario
for palabra, freq  in mi_vocabulario.items():
    if palabra not in tokens_mi_voc:
        tokens_mi_voc.append(palabra)


def filtrado(data_set_entrada, tokens_mi_voc):
    tokens_sublista=[]
    lista_filtrada=[]

    for sublista in data_set_entrada:
        for i in sublista:
            if i in tokens_mi_voc:
                if i not in tokens_sublista:
                    tokens_sublista.append(i)

        lista_filtrada.append(tokens_sublista)
        tokens_sublista=[]

    return lista_filtrada

#train_x ---> nueva lista de train 
train_x=filtrado(x_train,tokens_mi_voc)
#val_x ---> nueva lista de val 
val_x=filtrado(x_val,tokens_mi_voc)
#test_x ---> nueva lista de test 
test_x=filtrado(x_test,tokens_mi_voc)



# *********** Aquí termina la sección de agregar código *************


In [36]:
# Vemos el resultado de los primeros comentarios del conjunto de entrenamiento:

for ss in train_x[0:5]:
  print(ss)

['star', 'don', 'much', 'better', 'people', 'like', 'wasted']
['special', 'sucked']
['not', 'felt', 'server', 'terrible', 'job']
['call', 'steak', 'don', 'understand']
['however', 'sometimes', 'wrong', 'button']


#**Pregunta - 5:**


a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de Google y Glove de Stanford.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++


#**Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el “valor” será su vector embebido de dimensión 300.

Este diccionario deberá ser del mismo tamaño que el vocabulario previo que hayas construido previamente.

Es recomendable que una vez que generes el nuevo vocabulario de vectores embebidos, guardes dicho diccionario en un archivo.

Recuerda borrar la variable donde descargaste los 2 millones de vectores embebidos Fasttext.



In [37]:
#import fasttext
#Se guarda en una variable los todo el modelo 300 

#model_path="cc.en.300.bin"
#model_fasttext= fasttext.load_model(model_path)



In [38]:
#import fasttext
#import pickle
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

#Lista que guardara los tokens del vocabulario "tokens_mi_voc"
#mi_dict_fasttext={}
#for token in tokens_mi_voc:
 #   mi_dict_fasttext[token]=model_fasttext.get_word_vector(token)

#with open('mi_dict.pkl', 'wb') as archivo:
 #   pickle.dump(mi_dict_fasttext, archivo)

# *********** Aquí termina la sección de agregar código *************



#**Pregunta - 7:**

In [50]:
import pickle
with open('mi_diccionario.pkl', 'rb') as archivo:
    mi_dict_fast_text = pickle.load(archivo)

print(mi_dict_fast_text)

{'star': array([-2.86828041e-01,  1.15123533e-01,  5.38213179e-02,  6.26803637e-02,
        2.47128960e-02, -2.57342309e-02,  1.47269234e-01, -1.16852485e-01,
        1.43789779e-02,  2.02018499e-01, -6.00243136e-02, -6.75183982e-02,
       -4.77372715e-03,  4.89413664e-02,  1.16688116e-02, -3.23957461e-03,
       -1.23359617e-02, -8.51226449e-02, -4.92033921e-02,  6.35383427e-02,
        8.86741839e-03,  1.40952796e-01,  1.09847665e-01,  1.25429276e-02,
       -4.25020084e-02, -8.08417723e-02,  2.70867231e-03,  3.24510969e-02,
        1.41084492e-02,  1.06206566e-01,  1.08138800e-01, -5.41383438e-02,
        5.69636673e-02,  5.37327491e-02, -7.06092343e-02, -9.78406295e-02,
       -1.20484188e-01, -4.16465327e-02, -8.21356028e-02,  4.71536033e-02,
        9.90687776e-03,  4.89758067e-02, -1.33870468e-01, -1.54928997e-01,
       -1.81018680e-01,  1.78726465e-02,  7.79893203e-03, -8.59202892e-02,
       -2.29937509e-02, -1.81513757e-01, -1.04256801e-01,  1.19395129e-01,
        7.993146



Generamos los vectores embebidos a paertir de los conjuntos de entrenamiento, validación y preuba.

Los llamaremos trainEmb, valEmb y testEmb, respectivamente.

In [94]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********

import numpy as np
import pickle

def calcular_promedio_tokens(archivo_dict, set_entrada):
    """
    Calcula el promedio de los vectores de tokens para cada sublista en  train_val, val_train y test_train.

    Args:
    archivo_dict (str): Ruta del archivo pickle que contiene el diccionario de vectores.
    train_x (list): Lista de listas de tokens.

    Returns:
    list: Lista con los promedios de los vectores de tokens para cada sublista.
    """
    # Cargar el diccionario de vectores desde el archivo pickle
    with open(archivo_dict, 'rb') as archivo:
        mi_dict_fast_text = pickle.load(archivo)

    lista_promedio_tokens = []
    arreglos_acumulados_token=[]

    # Iterar sobre cada sublista de tokens en train_x
    for sublista_token in set_entrada:
        
        # Obtener el vector para cada token y agregarlo a arreglos_acumulados_token
        for token in sublista_token:
            vector_token = mi_dict_fast_text[token]
            arreglos_acumulados_token.append(vector_token)
        
        # Calcular el promedio de los vectores en arreglos_acumulados_token
        promedio_de_tokens = np.mean(np.array(arreglos_acumulados_token), axis=0)
        lista_promedio_tokens.append(promedio_de_tokens)
        promedio_de_tokens=[]
    
    return lista_promedio_tokens


# *********** Aquí termina la sección de agregar código *************

In [95]:
archivo_dict = 'mi_diccionario.pkl'
trainEmb= calcular_promedio_tokens(archivo_dict, train_x)
valEmb = calcular_promedio_tokens(archivo_dict, val_x)
testEmb = calcular_promedio_tokens(archivo_dict, test_x)

In [96]:
# Veamos las dimensiones de cada conjunto embebido:

print("Train-Emb:", pd.Series(trainEmb).shape)
print("Val-Emb:", pd.Series(valEmb).shape)
print("Test-Emb:", pd.Series(testEmb).shape)

Train-Emb: (2100,)
Val-Emb: (450,)
Test-Emb: (450,)


#**Pregunta - 8:**



Utiliza los modelos de regresión logística y bosque aleatorio (random forest) y encuentra sus desempeños.

Compara los resultados con los de la semana anterior.

In [ ]:
# REGRESIÓN LOGÍSTICA:

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Crear una instancia del modelo Logistic Regression
modelo_logistico = LogisticRegression()

# Entrenar el modelo con los datos de entrenamiento y pueba 

x_trainVal = np.concatenate((trainEmb, valEmb), axis=0)
y_trainVal = np.concatenate((y_train,y_val), axis=0)

modelo_logistico.fit(x_trainVal, y_trainVal)


# Hacer predicciones en el conjunto de prueba
predicciones_test = modelo_logistico.predict(testEmb)


reporte_test = classification_report(y_test, predicciones_test)
print("Informe de clasificación Datos (Modelo de Regresion)- Set de Prueba")
print(reporte_test)


# *********** Aquí termina la sección de agregar código *************


In [ ]:
# BOSQUE ALEATORIO (Random Forest):

# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********
from sklearn.ensemble import RandomForestClassifier

# Crear una instancia del modelo Random Forest
modelo_rf = RandomForestClassifier(class_weight="balanced")  # Puedes ajustar class_weight según sea necesario

# Entrenar el modelo con los datos de entrenamiento y pueba 

x_trainVal = np.concatenate((trainEmb, valEmb), axis=0)
y_trainVal = np.concatenate((y_train,y_val), axis=0)
modelo_rf.fit(x_trainVal, y_trainVal)



# Hacer predicciones en el conjunto de prueba
predicciones_test_rf = modelo_rf.predict(testEmb)

# Generar informe de clasificación para el conjunto de validación
reporte_val_rf = classification_report(y_val, predicciones_test_rf)
print("Informe de clasificación - Random Forest (Set de Validación):")
print(reporte_val_rf)

# Generar informe de clasificación para el conjunto de prueba
reporte_test_rf = classification_report(y_test, predicciones_test_rf)
print("Informe de clasificación - Random Forest (Set de Prueba):")
print(reporte_test_rf)



# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 9:**



Reporte del mejor modelo.


In [ ]:
# ******* Inlcuye a continuación todas las líneas de código y celdas que requieras: ***********


None


# *********** Aquí termina la sección de agregar código *************

#**Pregunta - 10:**



Incluye tus comentarios finales de la actividad.

### ++++++++ Inicia la sección de agregar texto: +++++++++++

None

### ++++++++ Termina la sección de agregar texto: +++++++++++

##**Fin de la Actividad de vectores Embebidos - FastText**